In [ ]:
#00 This code makes S-Final-1_1_model.vtk from S-Final-1_1.seg.nrrd (NOT RECOMMENDED BECAUSE POSITION, DIRECTION, DIMENSIONS OFF)

import SimpleITK as sitk
import vtk

# Load the NRRD segmentation
nrrd_file_path = "../Nate/S-Final-1_1.seg.nrrd"
segmentation = sitk.ReadImage(nrrd_file_path)

# Convert segmentation to VTK image data
vtk_data = vtk.vtkImageData()
vtk_data.SetDimensions(segmentation.GetSize())
vtk_data.SetSpacing(segmentation.GetSpacing())
vtk_data.SetOrigin(segmentation.GetOrigin())
vtk_data.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

vtk_array = vtk.vtkUnsignedCharArray()
vtk_array.SetVoidArray(sitk.GetArrayViewFromImage(segmentation).ravel(), segmentation.GetNumberOfPixels(), 1)
vtk_data.GetPointData().SetScalars(vtk_array)

# Create a Marching Cubes filter
marching_cubes = vtk.vtkMarchingCubes()
marching_cubes.SetInputData(vtk_data)
marching_cubes.SetValue(0, 1)  # Assuming Aseed is labeled with 1

# Smooth the surface
smoother = vtk.vtkWindowedSincPolyDataFilter()
smoother.SetInputConnection(marching_cubes.GetOutputPort())
smoother.SetNumberOfIterations(5)
smoother.BoundarySmoothingOff()
smoother.FeatureEdgeSmoothingOff()
smoother.SetFeatureAngle(120.0)
smoother.SetPassBand(0.001)
smoother.NonManifoldSmoothingOn()
smoother.NormalizeCoordinatesOn()
smoother.Update()

# Write the resulting model to VTK file
model_writer = vtk.vtkPolyDataWriter()
model_writer.SetInputConnection(smoother.GetOutputPort())
model_writer.SetFileName("../Nate/S-Final-1_1_model.vtk")
model_writer.Write()


In [ ]:
#00 This code displays '.nrrd' file in mayavie viewer (IGNORE)
import numpy as np
import SimpleITK as sitk
from mayavi import mlab
from mayavi.mlab import contour3d
def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

print("Imports and Function are good")
model_path = 'z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-2-MESAL-5010730-E3/Markups/88-Adam/S-Final-1_1.seg.nrrd'

# Load the .nrrd file using SimpleITK
reader = sitk.ImageFileReader()
reader.SetFileName(model_path)
image = reader.Execute()

#THIS STRIPS AWAY INFORMATION REAGARDING SPECIFIC POSITIONS
data = sitk.GetArrayFromImage(image)
transposed_data = np.transpose(data)
print("Image Transposed")

# Create a figure
mlab.figure(bgcolor=(0, 0, 0))

# Find Segmentation Center
model = sitk.ReadImage(model_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this

# Create an isosurface visualization of the segmentation
contour3d(transposed_data, colormap="hot", opacity = 1)
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
# Customize the visualization settings (colormap, opacity, etc.) as needed
print("Woohoo")
# Show the visualization
mlab.show()


In [ ]:
#00 This code shows the output model as one segmentation and save .png

#.png of 3D Model
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

from mayavi import mlab
import SimpleITK as sitk
import numpy as np

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# INPUTS: Aseed.vtk, Vseed.vitk, S-Final-1.seg.nrrd
# OUPUTS: .png file of model with wireframe mesh 

Vdecimated_file = "output_model(.01).vtk"
model_path= "S-Final-1_1.seg.nrrd"

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_filename = "../Nate/MODEL(codecreated-decimated).png"


mlab.figure(bgcolor=(1, 1, 1))
# srcA = mlab.pipeline.open(Adecimated_file)
srcV = mlab.pipeline.open(Vdecimated_file)

# Create a surface visualization of the data with blue color and 50% opacity
# surfaceA = mlab.pipeline.surface(srcA, color=(0, 0, 1), opacity=0.5)
# surface1A = mlab.pipeline.surface(srcA, color=(0, 0, 0))  # Blue color is (R=0, G=0, B=1)
surfaceV = mlab.pipeline.surface(srcV, color=(1, 0, 0), opacity=0.5)
surface1V = mlab.pipeline.surface(srcV, color=(0, 0, 0))  # Blue color is (R=0, G=0, B=1)

# # Show mesh faces in a different color (e.g., red) using wireframe representation
# surface1A.actor.property.representation = 'wireframe'
# surface1A.actor.property.line_width = 2  # You can adjust the line width
# surface1A.actor.property.color = (0, 0, 0)  # Red color is (R=1, G=0, B=0)
surface1V.actor.property.representation = 'wireframe'
surface1V.actor.property.line_width = 2  # You can adjust the line width
surface1V.actor.property.color = (0, 0, 0)  # Red color is (R=1, G=0, B=0)

#Find Center of Model
# model_path= "S-Segmentation-9.seg.nrrd"

model = sitk.ReadImage(model_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)

distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]

print(x)
print(y)
print(z)

ypt = y + 500
camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.view(azimuth=45, elevation=30, distance=400)
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

# magnification_factor = 5
# mlab.savefig(screenshot_filename, magnification=magnification_factor)

mlab.show()

 

In [ ]:
#01 THIS Code Makes Vseed.vtk from "S-Final-1_1.seg.nrrd"
import vtk

# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName("z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-49-Spiromics-67117583/Markups/88-Jensen/Models/S-Final-2(ONLYVSEED).seg.nrrd")
nrrd_reader.Update()

# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())

# Define scalar values for "Aseed" and "Vseed" segments
vseed_scalar_value = 1

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName("z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-49-Spiromics-67117583/Markups/88-Jensen/Models/Vseed.vtk")
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")


In [ ]:
#02 THIS Code Makes Aseed.vtk from "S-Final-1_1.seg.nrrd"

import vtk

# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName("z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-49-Spiromics-67117583/Markups/88-Jensen/Models/S-Final-2(ONLYVSEED).seg.nrrd")
# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())
# Define scalar values for "Aseed" and "Vseed" segments
aseed_scalar_value = 1
vseed_scalar_value = 2
# Create a threshold filter for "Aseed" segment
threshold_aseed = vtk.vtkImageThreshold()
threshold_aseed.SetInputData(vtk_image_data)
threshold_aseed.ThresholdBetween(aseed_scalar_value, aseed_scalar_value)
threshold_aseed.ReplaceInOn()
threshold_aseed.SetInValue(aseed_scalar_value)
threshold_aseed.ReplaceOutOn()
threshold_aseed.SetOutValue(0)
threshold_aseed.Update()
# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(threshold_aseed.GetOutput())
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()
# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName("z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-49-Spiromics-67117583/Markups/88-Jensen/Models/Aseed.vtk")
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()
print("Aseed Segmentation conversion complete.")


In [ ]:
#03 This code works to decimate both Aseed.vtk and Vseed.vtk (pyvista library)

import pyvista as pv

# Load the VTK mesh
mesh_Aseed = pv.read("../Nate/Models/Aseed.vtk")
mesh_Vseed = pv.read("../Nate/Models/Vseed.vtk")

# Perform decimation
Aseed_decimated_mesh = mesh_Aseed.decimate(target_reduction=0.99)  # Set the target reduction factor
Vseed_decimated_mesh = mesh_Vseed.decimate(target_reduction=0.99) 
# Save the decimated mesh

Aseed_decimated_mesh.save("../Nate/Models/Aseed(decimated1%).vtk")
Vseed_decimated_mesh.save("../Nate/Models/Vseed(decimated1%).vtk")
print("Decimation complete.")

In [ ]:
#03 This code works to decimate both Aseed.vtk and Vseed.vtk (vtk library)

import vtk

class VTKDecimator:
    def __init__(self, target_percentage=0.01):
        self.target_percentage = target_percentage

    def decimate_vtk(self, input_filename, output_filename):
        reader = vtk.vtkPolyDataReader()
        reader.SetFileName(input_filename)

        decimate = vtk.vtkQuadricDecimation()
        decimate.SetInputConnection(reader.GetOutputPort())
        decimate.SetTargetReduction(1 - self.target_percentage)
        decimate.Update()

        writer = vtk.vtkPolyDataWriter()
        writer.SetFileName(output_filename)
        writer.SetInputConnection(decimate.GetOutputPort())
        writer.Write()


# Change the target percentage here to change decimate factor
if __name__ == "__main__":
    decimator = VTKDecimator(target_percentage=0.01) #(0.01) = 99% decimation of original model

    aseed_input_filename = "../Nate/Models/Aseed.vtk"
    aseed_output_filename = "../Nate/Models/Aseed(decimated).vtk"
    # aseed_output_filename = "Aseed_decimated.vtk"
    decimator.decimate_vtk(aseed_input_filename, aseed_output_filename)

    vseed_input_filename = "../Nate/Models/Vseed.vtk"
    vseed_output_filename = "../Nate/Models/Vseed(decimated).vtk"
    decimator.decimate_vtk(vseed_input_filename, vseed_output_filename)


In [ ]:
#04 Screenshot of both Decimated models within mayavi viewer
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

from mayavi import mlab
import SimpleITK as sitk
import numpy as np

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# INPUTS: Aseed.vtk, Vseed.vitk, S-Final-1.seg.nrrd
# OUPUTS: .png file of model with wireframe mesh 

Adecimated_file = "../Nate/Models/ASeed(decimated).vtk"
Vdecimated_file = "../Nate/Models/VSeed(decimated).vtk"
model_path= "S-Final-1_1.seg.nrrd"

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_filename = "../Nate/Models/Model(Final).png"


mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(Adecimated_file)
srcV = mlab.pipeline.open(Vdecimated_file)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 0, 1), opacity=0.5)
surface1A = mlab.pipeline.surface(srcA, color=(0, 0, 0))  # Blue color is (R=0, G=0, B=1)
surfaceV = mlab.pipeline.surface(srcV, color=(1, 0, 0), opacity=0.5)
surface1V = mlab.pipeline.surface(srcV, color=(0, 0, 0))  # Blue color is (R=0, G=0, B=1)

# Show mesh faces in a different color (e.g., red) using wireframe representation
surface1A.actor.property.representation = 'wireframe'
surface1A.actor.property.line_width = 2  # You can adjust the line width
surface1A.actor.property.color = (0, 0, 0)  # Red color is (R=1, G=0, B=0)
surface1V.actor.property.representation = 'wireframe'
surface1V.actor.property.line_width = 2  # You can adjust the line width
surface1V.actor.property.color = (0, 0, 0)  # Red color is (R=1, G=0, B=0)

# Find Segmentation Center
model = sitk.ReadImage(model_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)

distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]

print(x)
print(y)
print(z)

ypt = y + 500
camera_focal_point = np.array([(x), (y), z])
camera_position = np.array([(x),(ypt),z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_filename, magnification=magnification_factor)

mlab.show()

 

In [ ]:
# FINAL WORKS TO MAKE DECIMATED MESH MODELS 
# THIS Code Makes Vseed.vtk from "S-Final-1_1.seg.nrrd" MISCELLANEOUS NEEDS TO BE DELETED. Aseed must be assigned a scalar value of 1 and Vseed must be assigned scalar value of 2
import vtk
import pyvista as pv
from mayavi import mlab
import SimpleITK as sitk
import numpy as np
import os

# Locate Folder for Final segmentation and change "segmentation_path" to match
# Create "output_path" for to save screenshot of final model to path that you specify
# Add the Case Number so that ".png" is saved correctly 

# segmentation_path= "../Nate/S-Final-1_1.seg.nrrd"

segmentation_path = 'z:/Q-Figures/T32-Poster/Nate/Case-49/S-Final-2(NOMISC).seg.nrrd'
case_number = "Case-49-Spiromics-67117583"
output_path = 'z:/Q-Figures/T32-Poster/Nate/Case-49/Models/'  # Change this to your desired output directory

# This line creates Models Forlder if not already made
if not os.path.exists(output_path):
    os.makedirs(output_path)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path + vtk_filename_aseed
output_filename_vseed = output_path + vtk_filename_vseed


#01 THIS Code Makes Vseed.vtk from "S-Final-1_1.seg.nrrd"
# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName(segmentation_path)
nrrd_reader.Update()

# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())

# Define scalar values for "Aseed" and "Vseed" segments
aseed_scalar_value = 1
vseed_scalar_value = 2

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")

#02 THIS Code Makes Aseed.vtk from "S-Final-1_1.seg.nrrd"
# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName(segmentation_path)
nrrd_reader.Update()

# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())

# Create a threshold filter for "Aseed" segment
threshold_aseed = vtk.vtkImageThreshold()
threshold_aseed.SetInputData(vtk_image_data)
threshold_aseed.ThresholdBetween(aseed_scalar_value, aseed_scalar_value)
threshold_aseed.ReplaceInOn()
threshold_aseed.SetInValue(aseed_scalar_value)
threshold_aseed.ReplaceOutOn()
threshold_aseed.SetOutValue(0)
threshold_aseed.Update()

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(threshold_aseed.GetOutput())
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation conversion complete.")

#03 This code works to decimate both Aseed.vtk and Vseed.vtk (pyvista library)
# import pyvista as pv

# Load the VTK mesh
mesh_Aseed = pv.read(output_filename_aseed)
mesh_Vseed = pv.read(output_filename_vseed)

# Perform decimation
Aseed_decimated_mesh = mesh_Aseed.decimate(target_reduction=0.99)  # Set the target reduction factor
Vseed_decimated_mesh = mesh_Vseed.decimate(target_reduction=0.99) 


vtk_model_aseed = f"{Aseed_model_base}.vtk"
vtk_model_vseed = f"{Vseed_model_base}.vtk"

output_model_aseed = output_path + vtk_model_aseed
output_model_vseed = output_path + vtk_model_vseed

# Save the decimated mesh
Aseed_decimated_mesh.save(output_model_aseed)
Vseed_decimated_mesh.save(output_model_vseed)
print("Aseed and Vseed Decimation complete.")

#04 Screenshot of both Decimated models within mayavi viewer
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

# from mayavi import mlab
# import SimpleITK as sitk
# import numpy as np

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# INPUTS: Aseed.vtk, Vseed.vitk, S-Final-1.seg.nrrd
# OUPUTS: .png file of model with wireframe mesh 

# output_filename_aseed = output_model_aseed
# output_filename_vseed = output_model_vseed

Adecimated_file = output_model_aseed
Vdecimated_file = output_model_vseed

Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"

vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
screenshot_filename = "-Decimated-Mesh-Model.png"

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path + case_number + screenshot_filename

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(Adecimated_file)
srcV = mlab.pipeline.open(Vdecimated_file)


# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surface1A = mlab.pipeline.surface(srcA, color=(0, 0, 0))  # Blue color is (R=0, G=0, B=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)
surface1V = mlab.pipeline.surface(srcV, color=(0, 0, 0))  # Blue color is (R=0, G=0, B=1)

# Show mesh faces in a different color (e.g., red) using wireframe representation
surface1A.actor.property.representation = 'wireframe'
surface1A.actor.property.line_width = 2  # You can adjust the line width
surface1A.actor.property.color = (0, 0, 0)  # Red color is (R=1, G=0, B=0)
surface1V.actor.property.representation = 'wireframe'
surface1V.actor.property.line_width = 2  # You can adjust the line width
surface1V.actor.property.color = (0, 0, 0)  # Red color is (R=1, G=0, B=0)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)

# Comment this line below out if you don't want to open 3D viewer
mlab.show()

print("Process Completed")



In [ ]:
# FINAL without wireframe and decimation
# THIS Code Makes Vseed.vtk from "S-Final-1_1.seg.nrrd" MISCELLANEOUS NEEDS TO BE DELETED
import vtk
import pyvista as pv
from mayavi import mlab
import SimpleITK as sitk
import numpy as np
import os

# Locate Folder for Final segmentation and change "segmentation_path" to match
# Create "output_path" for Final Model and ".png' to "Markups/Nate/Models"
# Add the Case Number so that ".png" is saved correctly 

# segmentation_path= "../Nate/S-Final-1_1.seg.nrrd"

segmentation_path = 'z:/Q-Figures/T32-Poster/Nate/Case-47/Case-47-S-Final-1(NOMISC).seg.nrrd'
case_number = "Case-47-Spiromics-65177004"
output_path = 'z:/Q-Figures/T32-Poster/Nate/Case-47/Models/'  # Change this to your desired output directory

# This line creates Models Forlder if not already made
if not os.path.exists(output_path):
    os.makedirs(output_path)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path + vtk_filename_aseed
output_filename_vseed = output_path + vtk_filename_vseed


#01 THIS Code Makes Vseed.vtk from "S-Final-1_1.seg.nrrd"
# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName(segmentation_path)
nrrd_reader.Update()

# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())

# Define scalar value for "Aseed" and "Vseed" segments
aseed_scalar_value = 2
vseed_scalar_value = 1

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")

#02 THIS Code Makes Aseed.vtk from "S-Final-1_1.seg.nrrd"
# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName(segmentation_path)
nrrd_reader.Update()

# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())

# Create a threshold filter for "Aseed" segment
threshold_aseed = vtk.vtkImageThreshold()
threshold_aseed.SetInputData(vtk_image_data)
threshold_aseed.ThresholdBetween(aseed_scalar_value, aseed_scalar_value)
threshold_aseed.ReplaceInOn()
threshold_aseed.SetInValue(aseed_scalar_value)
threshold_aseed.ReplaceOutOn()
threshold_aseed.SetOutValue(0)
threshold_aseed.Update()

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(threshold_aseed.GetOutput())
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation conversion complete.")


def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)


Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"

vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
screenshot_filename = "-Decimated-Mesh-Model.png"

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path + case_number + screenshot_filename

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(output_filename_aseed)
srcV = mlab.pipeline.open(output_filename_vseed)


# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)


# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)

# Comment this line below out if you don't want to open 3D viewer
mlab.show()

print("Process Completed")



In [ ]:
# CASE 46 Lung Mask and Aseed
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

from mayavi import mlab
import SimpleITK as sitk
import numpy as np

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# INPUTS: Aseed.vtk, Vseed.vitk, S-Final-1.seg.nrrd
# OUPUTS: .png file of model with wireframe mesh 

lungmask_file = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-47-0.5_QR40_RESEARCH_A_80KV_Qr40d_3_65177004/Markups/88-Nathan/Models/Models/Case-47-Lung-Mask-Model.vtk"
Adecimated_file = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-47-0.5_QR40_RESEARCH_A_80KV_Qr40d_3_65177004/Markups/88-Nathan/Models/Models/Case-47-Aseed.vtk"
model_path= "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-47-0.5_QR40_RESEARCH_A_80KV_Qr40d_3_65177004/Markups/88-Nathan/Case-47-S-Final-1(NOMISC).seg.nrrd"

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_filename = "z:/Q-Figures/T32-Poster/Nate/Case-47/Case-47-Lung-Mask-Aseed-Model.png"


mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(Adecimated_file)
srcV = mlab.pipeline.open(lungmask_file)

surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(128/255, 174/255, 128/255), opacity=.15)


# Find Segmentation Center
model = sitk.ReadImage(model_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)

distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]

print(x)
print(y)
print(z)

ypt = y + 550
camera_focal_point = np.array([(x), (y), z])
camera_position = np.array([(x),(ypt),z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_filename, magnification=magnification_factor)

mlab.show()

 

In [ ]:
# CASE 49 Lung Mask and Aseed
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

from mayavi import mlab
import SimpleITK as sitk
import numpy as np

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

# INPUTS: Aseed.vtk, Vseed.vitk, S-Final-1.seg.nrrd
# OUPUTS: .png file of model with wireframe mesh 

Vdecimated_file = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-46-SPIROMICS-51186800/Markups/88-Jensen/Models/Case-46-Lung-Mask.vtk"
Adecimated_file = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-46-SPIROMICS-51186800/Markups/88-Jensen/Models/Case-46-Aseed.vtk"
model_path= "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-46-SPIROMICS-51186800/Markups/88-Jensen/Models/Case-46-S-Final-2(NOMISC).nrrd"

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_filename = "z:/Q-Figures/T32-Poster/Nate/Case-46/Case-46-Lung-Mask-Aseed-Model.png"


mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(Adecimated_file)
srcV = mlab.pipeline.open(Vdecimated_file)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)

surfaceV = mlab.pipeline.surface(srcV, color=(128/255, 174/255, 128/255), opacity=.15)


# Find Segmentation Center
model = sitk.ReadImage(model_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)

distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]

print(x)
print(y)
print(z)

ypt = y + 550
camera_focal_point = np.array([(x), (y), z])
camera_position = np.array([(x),(ypt),z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_filename, magnification=magnification_factor)

mlab.show()

 

In [ ]:
# FINAL without wireframe and decimation
# THIS Code Makes Vseed.vtk from "S-Final-1_1.seg.nrrd" MISCELLANEOUS NEEDS TO BE DELETED
import vtk
import pyvista as pv
from mayavi import mlab
import SimpleITK as sitk
import numpy as np
import os

# Locate Folder for Final segmentation and change "segmentation_path" to match
# Create "output_path" for Final Model and ".png' to "Markups/Nate/Models"
# Add the Case Number so that ".png" is saved correctly 

# segmentation_path= "../Nate/S-Final-1_1.seg.nrrd"

segmentation_path = 'z:/Q-Figures/T32-Poster/Nate/Case-47/Case-47-S-Final-1(NOMISC).seg.nrrd'
case_number = "Case-47-Spiromics-65177004"
output_path = 'z:/Q-Figures/T32-Poster/Nate/Case-47/Models/'  # Change this to your desired output directory

# This line creates Models Forlder if not already made
if not os.path.exists(output_path):
    os.makedirs(output_path)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path + vtk_filename_aseed
output_filename_vseed = output_path + vtk_filename_vseed


#01 THIS Code Makes Vseed.vtk from "S-Final-1_1.seg.nrrd"
# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName(segmentation_path)
nrrd_reader.Update()

# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())

# Define scalar value for "Aseed" and "Vseed" segments
aseed_scalar_value = 1
vseed_scalar_value = 2

# Create a Marching Cubes filter for "Vseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data)
contour_aseed.SetValue(0, vseed_scalar_value)
contour_aseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_vseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Vseed Segmentation conversion complete.")

#02 THIS Code Makes Aseed.vtk from "S-Final-1_1.seg.nrrd"
# Load the .nrrd segmentation
nrrd_reader = vtk.vtkNrrdReader()
nrrd_reader.SetFileName(segmentation_path)
nrrd_reader.Update()

# Convert to a VTK image data object
vtk_image_data = vtk.vtkImageData()
vtk_image_data.DeepCopy(nrrd_reader.GetOutput())

# Create a threshold filter for "Aseed" segment
threshold_vseed = vtk.vtkImageThreshold()
threshold_vseed.SetInputData(vtk_image_data)
threshold_vseed.ThresholdBetween(vseed_scalar_value, vseed_scalar_value)
threshold_vseed.ReplaceInOn()
threshold_vseed.SetInValue(vseed_scalar_value)
threshold_vseed.ReplaceOutOn()
threshold_vseed.SetOutValue(0)
threshold_vseed.Update()

# Create a Marching Cubes filter for "Aseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(threshold_vseed.GetOutput())
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Aseed Segmentation conversion complete.")



def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"

vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
screenshot_filename = "-Decimated-Mesh-Model.png"

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path + case_number + screenshot_filename

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(output_filename_aseed)
srcV = mlab.pipeline.open(output_filename_vseed)


# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)

# Comment this line below out if you don't want to open 3D viewer
mlab.show()

print("Process Completed")

